In [28]:
import os
import pdfplumber
from docx import Document
import spacy
from spacy import displacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")


In [29]:

nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def save_text_to_file(text, output_file):
    with open(output_file, "w", encoding="utf-8") as file:
        file.write(text)

def main(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        input_file = os.path.join(input_folder, filename)
        output_file = os.path.join(output_folder, os.path.splitext(filename)[0] + ".txt")
        
        if filename.endswith(".pdf"):
            text = extract_text_from_pdf(input_file)
        elif filename.endswith(".docx"):
            text = extract_text_from_docx(input_file)
        else:
            print(f"Ignoring file: {filename}. Unsupported format.")
            continue
        
        # Perform any additional processing using spaCy if needed
        # Example: doc = nlp(text)

        save_text_to_file(text, output_file)
        print(f"Converted {input_file} to {output_file}")

if __name__ == "__main__":
    input_folder = "C:\\Users\\anubh\\Downloads\\Slides_for_topics"
  # Specify the directory containing your input PDFs or DOCX files
    output_folder = "C:\\Users\\anubh\\Downloads\\Output"  # Specify the directory where you want to save the output text files
    main(input_folder, output_folder)

Converted C:\Users\anubh\Downloads\Slides_for_topics\Algorithm analysis.pdf to C:\Users\anubh\Downloads\Output\Algorithm analysis.txt
Converted C:\Users\anubh\Downloads\Slides_for_topics\Exception Handelling.pdf to C:\Users\anubh\Downloads\Output\Exception Handelling.txt
Converted C:\Users\anubh\Downloads\Slides_for_topics\Exception Handelling_Tutorial.pdf to C:\Users\anubh\Downloads\Output\Exception Handelling_Tutorial.txt
Converted C:\Users\anubh\Downloads\Slides_for_topics\Graphs.pdf to C:\Users\anubh\Downloads\Output\Graphs.txt
Converted C:\Users\anubh\Downloads\Slides_for_topics\Graphs_Tutorial.pdf to C:\Users\anubh\Downloads\Output\Graphs_Tutorial.txt
Converted C:\Users\anubh\Downloads\Slides_for_topics\Hashmap.pdf to C:\Users\anubh\Downloads\Output\Hashmap.txt
Converted C:\Users\anubh\Downloads\Slides_for_topics\Hashmap_Tutorial.pdf to C:\Users\anubh\Downloads\Output\Hashmap_Tutorial.txt
Converted C:\Users\anubh\Downloads\Slides_for_topics\Intelligent Agents.pdf to C:\Users\anub

In [30]:
import os
import spacy
import requests

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define the directory containing the processed text files
directory = "C:\\Users\\anubh\\Downloads\\Output"

# Define the API endpoint and headers
url = "https://api.dbpedia-spotlight.org/en/annotate"
headers = {"accept": "application/json"}
ai_ml_keywords = {
    "artificial", "intelligence", "ai", "machine", "learning", "ml", "intelligent", "system",
    "programming", "algorithm", "deep", "neural", "network", "data", "analysis", "prediction",
    "knowledge", "graphs", "text", "mining", "rdf", "content-based", "recommendations",
    "collaborative", "filtering", "k-means", "clustering", "pattern-matching", "bots",
    "natural", "language", "processing", "nlp", "inheritance", "tail", "recursion",
    "efficiency", "hash", "table", "run", "time", "errors", "binary", "trees", "depth",
    "first", "search", "dfs"
}

# Function to filter named entities based on POS tags
def filter_named_entities(doc):
    named_entities = []
    for ent in doc.ents:
        if ent.label_ in ["PERSON", "ORG", "GPE", "LOC", "EVENT", "WORK_OF_ART", "PRODUCT"]:
            named_entities.append(ent.text)
    return named_entities

# Set to store seen URIs
seen_uris = set()

# Output file path
output_file_path = "output.txt"

# Open the output file in append mode
with open(output_file_path, "w", encoding="utf-8") as output_file:
    # Iterate over each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            # Read the processed document from the file with specified encoding
            filepath = os.path.join(directory, filename)
            with open(filepath, "r", encoding="utf-8") as file:
                document_text = file.read()

            # Process the document with spaCy
            doc = nlp(document_text)

            # Filter named entities
            named_entities = filter_named_entities(doc)

            # Iterate over named entities and make requests to DBpedia Spotlight for each entity
            for entity in named_entities:
                # Make the GET request to DBpedia Spotlight
                params = {"text": entity}
                response = requests.get(url, params=params, headers=headers)

                # Check if the request was successful
                if response.status_code == 200:
                    # Parse the JSON response
                    data = response.json()
                    # Write the annotations to the output file if not seen before
                    if "Resources" in data:
                        for resource in data["Resources"]:
                            if resource["@URI"] not in seen_uris:
                                # Print to console
                                print("File:", filename)
                                print("Entity:", entity)
                                print("URI:", resource["@URI"])
                                print("Surface Form:", resource["@surfaceForm"])
                                print("Types:", resource["@types"])
                                print()
                                # Write to file
                                output_file.write("File: " + filename + "\n")
                                output_file.write("Entity: " + entity + "\n")
                                output_file.write("URI: " + resource["@URI"] + "\n")
                                output_file.write("Surface Form: " + resource["@surfaceForm"] + "\n")
                                output_file.write("Types: " + resource["@types"] + "\n\n")
                                seen_uris.add(resource["@URI"])  # Add URI to seen URIs set
                else:
                    # Print error to console
                    print(f"Error for entity {entity} in file {filename}:", response.status_code)
                    # Write error to file
                    output_file.write(f"Error for entity {entity} in file {filename}: {response.status_code}\n")

# Print confirmation message
print("Output has been written to:", output_file_path)


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 79: character maps to <undefined>

In [53]:
import csv

# Define a function to format the URI
def format_uri(entity):
    return "<http://dbpedia.org/resource/{}>".format(entity.replace(" ", "_"))

# Define a function to format the TTL entry with additional properties
def format_ttl(entity, surface_form, uri, course, lecture):
    return """
    # Subject-Predicate-Object Triple
    {} a ex:Topic ;
        ex:TopicName "{}" ;
        ex:dbpediaLink {} ;
        ex:provenance "Algorithm analysis" ;
        ex:topic_in_course {} ;
        ex:topic_in_lecture {} .
    """.format(uri, surface_form, uri, course, lecture)

# Initialize an empty string to store the TTL content
ttl_content = ""

# Prefix Declarations
prefixes = """
@prefix ex: <http://example.org/> .
@prefix dbpedia: <http://dbpedia.org/resource/> .
"""

# Write the prefix declarations to the TTL content
ttl_content += prefixes

# Create a dictionary to store lecture-course mappings
lecture_course_map = {}

# Read the course info from the CSV file
with open("Topic_information.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        lecture_course_map[row['Lecture name']] = row['Course']

# Open the input file and read line by line
with open("C:\\Users\\anubh\\Downloads\\a.txt", "r") as file:
    lines = file.readlines()

# Initialize variables to store entity, URI, surface form, and lecture
entity = ""
uri = ""
surface_form = ""
lecture = ""

# Iterate through the lines of the input file
for line in lines:
    # Remove leading/trailing whitespaces and check if the line is not empty
    line = line.strip()
    if line:
        # Extract entity, URI, and surface form
        if line.startswith("Entity:"):
            entity = line.split("Entity: ")[1]
        elif line.startswith("URI:"):
            uri = line.split("URI: ")[1]
        elif line.startswith("Surface Form:"):
            surface_form = line.split("Surface Form: ")[1]
        elif line.startswith("File:"):
            lecture = line.split("File: ")[1].strip()[:-4]  # Remove the last 4 characters (".txt")
            # Check if the lecture exists in the mapping
            if lecture in lecture_course_map:
                course = lecture_course_map[lecture]
            else:
                # Handle the case when the lecture is not found in the mapping
                print(f"Error: Lecture '{lecture}' not found in the course mapping.")
                continue
        # Check if entity, URI, surface form, and lecture are all non-empty
        if entity and uri and surface_form and lecture:
            # Format and append TTL entry to the TTL content
            ttl_content += format_ttl(entity, surface_form, format_uri(entity), course, lecture)
            # Reset entity, URI, surface form, and lecture
            entity = ""
            uri = ""
            surface_form = ""
            lecture = ""

# Write the TTL content to an output file
with open("output.ttl", "w") as file:
    file.write(ttl_content)


In [ ]:
import requests
C:\\Users\\anubh\\Downloads\\a.txt
# Define the API endpoint and parameters
url = "https://api.dbpedia-spotlight.org/en/annotate"
params = {
    "text": "Concordia advanced six spots to 10th place among Canada’s engineering schools in the Maclean’s 2018 Program Rankings, while computer science advanced three spots into 11th position this year."
}
headers = {
    "accept": "application/json"
}

# Make the GET request
response = requests.get(url, params=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Print the annotations
    if "Resources" in data:
        for resource in data["Resources"]:
            print("URI:", resource["@URI"])
            print("Surface Form:", resource["@surfaceForm"])
            print("Types:", resource["@types"])
            print()
else:
    print("Error:", response.status_code)
